In [1]:
# Author: Mike Hagenow
# Generate some templated behaviors for sanding (including corrections) to be learned through DMP-LWR

import numpy as np
import rospy

from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import HybridSegment
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import DMPLWRhardcoded
from core_robotics.PyBSpline import BSplineSurface
from scipy.spatial.transform import Rotation as ScipyR
from scipy.spatial.transform import Slerp

# starting vals is a list of starting vals
# ending vals is a list of ending vals
# num_pts is how many interp values
# quat_vars is the index of any starts of quaternions so they can be SLERPed instead of LERPed
# super_pos_vars is the starting index for 2-pairs of variables which should have an added sine wave (e.g., rubbing)
# super_pos_freq is the frequency for the starting index pairs
# super_pos_amp is the amplitude for the starting index pairs
def interpMultD(starting_vals,ending_vals,num_pts,quat_vars=[], super_pos_vars = [], super_pos_freq=[], super_pos_amp = []):
    vals = []
    
    # LERP ALL VARIABLES
    for ii in range(0,num_pts):
       
        # super is to potentially add a circular motion on top of the LERP motion
        freq_super = 0.0
        amp_super = 0.0
        added_sine = np.zeros((np.shape(starting_vals)))
        for jj in range(0,len(super_pos_vars)):
            amp = super_pos_amp[jj]
            freq = super_pos_freq[jj]
            added_sine[super_pos_vars[jj]] = np.sin(np.pi*float(ii)/num_pts)*amp*np.sin(freq*float(ii)/2*np.pi)
            added_sine[super_pos_vars[jj]+1] = np.sin(np.pi*float(ii)/num_pts)*amp*np.cos(freq*float(ii)/2*np.pi)
        c_i = float(ii)/float(num_pts) # interpolation coefficient
        vals.append(list(c_i*(e-s)+s+a for s,e,a in zip(starting_vals,ending_vals,added_sine)))
    
    vals = np.asarray(vals) # currently,num_samples x num_vars
    
    # SLERP QUATERIONS
    for jj in range(0,len(quat_vars)):
        starting_q = np.array(starting_vals[quat_vars[jj]:quat_vars[jj]+4])
        ending_q = np.array(ending_vals[quat_vars[jj]:quat_vars[jj]+4])
        key_times = [0,num_pts-1]
        key_rots = ScipyR.from_quat(np.concatenate((starting_q.reshape((1,4)),ending_q.reshape((1,4))),axis=0))
        slerper = Slerp(key_times, key_rots)
        
        pts = list(range(0,num_pts))
        slerped = slerper(pts)
        vals[:,quat_vars[jj]:quat_vars[jj]+4] = np.asarray(slerped.as_quat())
    
    return vals.T

# returns num_vars x num_samples


# starting_vals = np.array([0, 1, 2, 0, 0, 0, 1])
# ending_vals = np.array([1, 2, 3, 0, 0, 0, -1])
# out = interpMultD(starting_vals,ending_vals,10,quat_vars=[3])

# print(out)




In [2]:
########################################
# The Sandening                       ##
########################################

starting_surf = [0.1, 0.2]
ending_surf = [0.1, 0.8]
num_passes = 1

samps_per_sec = 40

import rospkg
rospack = rospkg.RosPack()
uliconfigpath = rospack.get_path('uli_config')


surfaceBSplineFile = uliconfigpath+'/registration_models/IRC_piece1.csv'
surfaceBSpline = BSplineSurface()
surfaceBSpline.loadSurface(surfaceBSplineFile)

segments = []

from scipy.spatial.transform import Rotation as ScipyR
ang_relative_to_surface = 90 # degrees
R_tool_surf = ScipyR.from_euler('ZY',[ang_relative_to_surface,5],degrees=True)

q_ts = R_tool_surf.as_quat()
print(q_ts)

# Part 1: approach the starting point
segment1 = HybridSegment()
segment1.hybrid = False
segment1.num_samples = 6*samps_per_sec # 2 seconds
segment1.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment1.corrections.append(np.zeros((len(segment1.state_names),segment1.num_samples)))

approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(starting_surf[0], starting_surf[1])
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_app = (R_surf * R_tool_surf).as_quat()
if q_app[3]<0:
    q_app*=-1.0

print("QAPP: ",q_app)

starting = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.005, q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 1.0]
orig_vals = interpMultD(starting,ending,segment1.num_samples,quat_vars=[3])


# number of state variables by number of samples

segment1.original_vals = []
segment1.original_vals.append(orig_vals)

segments.append(segment1)


# Part 2: one pass over surface
segment2 = HybridSegment()
segment2.hybrid = True
segment2.surface = surfaceBSpline
segment2.num_samples = 10*samps_per_sec # 8 seconds
segment2.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw','delta_s','valve']
segment2.original_vals = []

R_extra_tool = ScipyR.from_euler('x',-15,degrees=True)
q_extra = R_extra_tool.as_quat()
corrections = [0.0, 0.0, -5.0, q_extra[0], q_extra[1], q_extra[2], q_extra[3], -0.5, 0.0]
segment2.corrections.append(interpMultD(corrections,corrections,segment2.num_samples,quat_vars=[3]))
# segment2.corrections.append(np.zeros((len(segment2.state_names),segment2.num_samples)))

samps = int(segment2.num_samples / 5)
app_force = -10.0
amp = 0.0
freq = 0.0

waypoint_0 = interpMultD([starting_surf[0], starting_surf[1], app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[ending_surf[0],ending_surf[1], app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],segment2.num_samples,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
segment2.original_vals.append(waypoint_0)

segments.append(segment2)


# # Part 3: move away from the surface
segment3 = HybridSegment()
segment3.hybrid = False
segment3.num_samples = 2*samps_per_sec # 2 seconds
segment3.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment3.original_vals = []
segment3.corrections.append(np.zeros((len(segment3.state_names),segment3.num_samples)))

retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(ending_surf[0], ending_surf[1])
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_ret = (R_surf * R_tool_surf).as_quat()

starting = [retract_pt[0], retract_pt[1], retract_pt[2]+ 0.005, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 1.0]
ending = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
orig_vals = interpMultD(starting,ending,segment3.num_samples,quat_vars=[3])
segment3.original_vals.append(orig_vals)

segments.append(segment3)


# # Part 4: move from end to start of next pass
segment4 = HybridSegment()
segment4.hybrid = False
segment4.num_samples = 4*samps_per_sec # 4 seconds
segment4.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment4.original_vals = []
segment4.corrections.append(np.zeros((len(segment4.state_names),segment4.num_samples)))

retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(ending_surf[0], ending_surf[1])
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_ret = (R_surf * R_tool_surf).as_quat()
if q_ret[3]<0:
    q_ret*=-1.0

starting = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
orig_vals = interpMultD(starting,ending,segment4.num_samples,quat_vars=[3])
segment4.original_vals.append(orig_vals)

segments.append(segment4)

for ii in range(0,num_passes-2):
    segments.extend(segments[0:4])
segments.extend(segments[:3])

[-0.03084356  0.03084356  0.70643377  0.70643377]
QAPP:  [ 3.81577190e-04 -1.56225505e-01  9.95512796e-03  9.87671171e-01]


In [ ]:
############################
# Learn IRC1 model         #
############################
import rospkg
rospack = rospkg.RosPack()
config_dir = rospack.get_path('uli_config')+'/registration_models/'


model = DMPLWRhardcoded(verbose=True, dt=1/samps_per_sec)
learnedSegments = model.learnModel(segments,config_dir+'IRC_piece1.pkl') # second argument is the outfile


-------------------
Model Learning: DMP + LWR Hardcoded
-------------------
DMP for segment  0  of  6

DMP for segment  1  of  6


In [6]:
############################
# Execute IRC1             #
############################

rospy.init_node('executeROSfromState', anonymous=True)

r_surf = np.array([0.4091875321237706, 0.3659806127623546, 0.6255911510276543, -0.5543099010413256])
t_surf = np.array([0.34226860621143673, -0.22084696598873957, 0.281413347966159])

model.executeModel(learnedSegments=learnedSegments, R_surface = r_surf, t_surface = t_surf, input_type='1dof')

not hybrid
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: 

deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [0. 0. 0.]


rotvec: [-0.26  0.    0.  ]
deltas: 0.5134178344097533
rofilt: [-0.19  0.    0.  ]
rotvec: [-0.26  0.    0.  ]
deltas: 0.5113703926967348
rofilt: [-0.19  0.    0.  ]
rotvec: [-0.26  0.    0.  ]
deltas: 0.5096731522462912
rofilt: [-0.2  0.   0. ]
rotvec: [-0.26  0.    0.  ]
deltas: 0.5082656761374176
rofilt: [-0.2  0.   0. ]
rotvec: [-0.26  0.    0.  ]
deltas: 0.5070981222806141
rofilt: [-0.2  0.   0. ]
rotvec: [-0.02  0.    0.  ]
deltas: 0.5061293360219746
rofilt: [-0.21  0.    0.  ]
rotvec: [-0.  0.  0.]
deltas: 0.5306954784459893
rofilt: [-0.21  0.    0.  ]
rotvec: [-0.  0.  0.]
deltas: 0.5761163264712204
rofilt: [-0.2  0.   0. ]
rotvec: [-0.  0.  0.]
deltas: 0.6303863808738555
rofilt: [-0.2  0.   0. ]
rotvec: [-0.  0.  0.]
deltas: 0.685887081931539
rofilt: [-0.19  0.    0.  ]
rotvec: [-0.  0.  0.]
deltas: 0.7381821427688007
rofilt: [-0.18  0.    0.  ]
rotvec: [-0.  0.  0.]
deltas: 0.7849447240671578
rofilt: [-0.17  0.    0.  ]
rotvec: [-0.  0.  0.]
deltas: 0.8252763804478185
rofilt:

rotvec: [-0.  0.  0.]
deltas: 1.019579808611645
rofilt: [0.05 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0153918625411904
rofilt: [0.04 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0121044781438013
rofilt: [0.04 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0095221760728366
rofilt: [0.03 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0074926013434935
rofilt: [0.03 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0058967483606696
rofilt: [0.03 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0046414990612644
rofilt: [0.02 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.003653891275684
rofilt: [0.02 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0028766937895155
rofilt: [0.02 0.   0.  ]
rotvec: [-0.01  0.    0.  ]
deltas: 1.0022649762318387
rofilt: [0.02 0.   0.  ]
rotvec: [-0.26  0.    0.  ]
deltas: 1.0006504354090553
rofilt: [0.01 0.   0.  ]
rotvec: [-0.26  0.    0.  ]
deltas: 0.952893793909815
rofilt: [-0.01  0.    0.  ]
rotvec: [-0.26  0.    0.  ]
deltas: 0.8828724534594742
rofilt: [-0.02  0.    0.  ]
rotvec: [-0

rotvec: [-0.  0.  0.]
deltas: 1.001561363304093
rofilt: [0.01 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0012294988625678
rofilt: [0.01 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0009682019564747
rofilt: [0.01 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0007624555615542
rofilt: [0.01 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0006004427240383
rofilt: [0.01 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0004728629696307
rofilt: [0.01 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0003723953723138
rofilt: [0.01 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.000293276578466
rofilt: [0.01 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: 1.0002309690260098
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0001818999876366
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0001432562623054
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0001128226126732
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000888546053015
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000699785143246
rofilt: [0. 0. 0.]
ro

...

rotvec: [-0.  0.  0.]
deltas: -1.1981321365745718
rofilt: [0.15 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: -1.1534005821141864
rofilt: [0.13 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: -1.1191564036540915
rofilt: [0.12 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: -1.0928198222442815
rofilt: [0.11 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: -1.0724721462840627
rofilt: [0.09 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: -1.056689752714952
rofilt: [0.08 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: -1.0444091003863751
rofilt: [0.07 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: -1.0348287405054426
rofilt: [0.06 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: -1.0273397612069277
rofilt: [0.06 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: -1.02147625411498
rofilt: [0.05 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: -1.0168796381304583
rofilt: [0.04 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: -1.0132726219857728
rofilt: [0.04 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas: -1.0104399575845315
rofilt: [0.03 0.   0.  ]
rotvec: [-0.  0.  0.]
deltas

rotvec: [-0.  0.  0.]
deltas: 1.0000000000000198
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000000000000155
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000000000000122
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000000000000098
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000000000000075
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.000000000000006
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000000000000047
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000000000000038
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000000000000029
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000000000000022
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000000000000018
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000000000000013
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.000000000000001
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000000000000009
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0000000000000007
r

rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 

[WARN] [1652971654.469618]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 0. 0.]
rotvec: [-0.  0.  0.]
deltas: 1.0
rofilt: [0. 

In [ ]:
# ########################################
# # IRC BEHAVIOR 1 - with superposition ##
# ########################################

# # Behavior three: Three passes on the IRC surface (slow with corrections)

# import rospkg
# rospack = rospkg.RosPack()
# uliconfigpath = rospack.get_path('uli_config')


# surfaceBSplineFile = uliconfigpath+'/registration_models/IRC_piece1.csv'
# surfaceBSpline = BSplineSurface()
# surfaceBSpline.loadSurface(surfaceBSplineFile)

# segments = []

# from scipy.spatial.transform import Rotation as ScipyR
# ang_relative_to_surface = 90 # degrees
# R_tool_surf = ScipyR.from_euler('z',ang_relative_to_surface,degrees=True)
# q_ts = R_tool_surf.as_quat()


# # Part 1: approach the starting point
# segment1 = HybridSegment()
# segment1.hybrid = False
# segment1.num_samples = 600 # 2 seconds
# segment1.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
# segment1.corrections.append(np.zeros((len(segment1.state_names),segment1.num_samples)))

# approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.1, 0.2)
# R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
# q_app = (R_surf * R_tool_surf).as_quat()
# if q_app[3]<0:
#     q_app*=-1.0

# print("QAPP: ",q_app)

# starting = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
# ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.01, q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 1.0]
# orig_vals = interpMultD(starting,ending,segment1.num_samples,quat_vars=[3])


# # number of state variables by number of samples

# segment1.original_vals = []
# segment1.original_vals.append(orig_vals)

# segments.append(segment1)


# # Part 2: one pass over surface
# segment2 = HybridSegment()
# segment2.hybrid = True
# segment2.surface = surfaceBSpline
# segment2.num_samples = 2000 # 8 seconds
# segment2.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw','delta_s','valve']
# segment2.original_vals = []

# R_extra_tool = ScipyR.from_euler('y',5,degrees=True)
# q_extra = R_extra_tool.as_quat()
# corrections = [0.0, 0.0, -5.0, q_extra[0], q_extra[1], q_extra[2], q_extra[3], -1.5, 0.0]
# segment2.corrections.append(interpMultD(corrections,corrections,segment2.num_samples,quat_vars=[3]))


# samps = int(segment2.num_samples / 5)
# app_force = -8.0
# amp = 0.05
# freq = 0.2
# waypoint_0 = interpMultD([0.1, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.1, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# waypoint_1 = interpMultD([0.1, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.5, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# waypoint_2 = interpMultD([0.5, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.5, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# waypoint_3 = interpMultD([0.5, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.9, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# waypoint_4 = interpMultD([0.9, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.9, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])

# waypoints = np.hstack([waypoint_0,waypoint_1,waypoint_2,waypoint_3,waypoint_4])

# segment2.original_vals.append(waypoints)

# segments.append(segment2)


# # # Part 3: move away from the surface
# segment3 = HybridSegment()
# segment3.hybrid = False
# segment3.num_samples = 200 # 2 seconds
# segment3.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
# segment3.original_vals = []
# segment3.corrections.append(np.zeros((len(segment3.state_names),segment3.num_samples)))

# retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.9, 0.5)
# R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
# q_ret = (R_surf * R_tool_surf).as_quat()

# starting = [retract_pt[0], retract_pt[1], retract_pt[2]+ 0.01, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 1.0]
# ending = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
# orig_vals = interpMultD(starting,ending,segment3.num_samples,quat_vars=[3])
# segment3.original_vals.append(orig_vals)

# segments.append(segment3)


# # # Part 4: move from end to start of next pass
# segment4 = HybridSegment()
# segment4.hybrid = False
# segment4.num_samples = 400 # 4 seconds
# segment4.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
# segment4.original_vals = []
# segment4.corrections.append(np.zeros((len(segment4.state_names),segment4.num_samples)))

# retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.9, 0.5)
# R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
# q_ret = (R_surf * R_tool_surf).as_quat()
# if q_ret[3]<0:
#     q_ret*=-1.0

# starting = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
# ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
# orig_vals = interpMultD(starting,ending,segment4.num_samples,quat_vars=[3])
# segment4.original_vals.append(orig_vals)

# segments.append(segment4)

# segments.extend(segments)
# segments.extend(segments[:3])